# Get dates

In [1]:
import pandas as pd
import requests
import csv
import numpy as np
from collections import OrderedDict


In [2]:
df = pd.read_pickle('/Users/anyway/Desktop/ngrams/norm-unigrams_r.pkl')
df.reset_index(inplace=True)

In [3]:
df.head()

,unigram,year,page_count,volume_count
0,р'1ас,1970,7,1
1,р'1ас,1973,2,2
2,р'1ас,1976,5,5
3,р'1ас,1977,52,7
4,р'1ас,1978,20,7


In [4]:
df["unigram"].nunique()

84765

In [5]:
unique = df["unigram"].unique()
unique = unique.tolist()

### Cоздадим словарь униграмм

In [6]:
# сначала прочтем нормализованные Викиданные из файла
def read_tsv(file):
    with open(file, "r", encoding="UTF-8") as f:
        f = f.readlines()
        print(len(f))
    data = []
    for line in f:
        line = line.strip('\n').split(' | ')
        data.append(line)
    return data

norm_data = read_tsv("/Users/anyway/Desktop/wikidata/norm-wikidata.tsv")

2234352


In [7]:
norm_data[:10]

[['Q31', 'бельгия', 'королевство бельгия'],
 ['Q8', 'счастие'],
 ['Q23', 'джордж вашингтон', 'вашингтон джордж'],
 ['Q24', 'джек бауэр'],
 ['Q42', 'дуглас адамс', 'адамс дуглас'],
 ['Q1868', 'поль отль', 'отль поль'],
 ['Q2013', 'викидать', 'викидат'],
 ['Q45', 'португалия', 'португальский республика'],
 ['Q51', 'антарктида'],
 ['Q58', 'пенис', 'детородный орган', 'половой член', 'член']]

In [8]:
def update_dict(dict_, ngrams, Q): # функция обновляет словарь нграм 
    
    if isinstance(ngrams, str):
        ngram = ngrams
        if ngram not in dict_:
            dict_[ngram] = {Q}
        else:
            dict_[ngram].add(Q)

    if isinstance(ngrams, list):
        for ngram in ngrams:
            if ngram not in dict_:
                dict_[ngram] = {Q}
            else:
                dict_[ngram].add(Q)
            
    return 0

In [ ]:
# создадим словарь униграмм

# не знаю, стоит ли делить биграммы также на униграммы и дополнить ими словарь. Пока решила так не делать.  

unigram_dict = {}
for line in norm_data:
    if len(line[1:]) == 1 and len(line[1].split()) == 1:
        Q, unigram = line[0], line[1]
        update_dict(unigram_dict, unigram, Q)
    elif len(line[1:]) > 1:
        for entity_name in line[1:]:
            entity_name_split = entity_name.split()
            if len(entity_name_split) == 1:
                Q, unigram = line[0], entity_name_split[0]
                update_dict(unigram_dict, unigram, Q)
len(unigram_dict)

### Найдем соответствия между униграмами и сущностями в Викиданных

In [10]:
unigrams_with_Q = {}
for word in unique:
    if word in unigram_dict:
        unigrams_with_Q[word] = unigram_dict[word]
        
len(unigrams_with_Q)

7152

Итак, мы будем строить минимум 7 тысяч запросов на SPARQL (одному запросу может соответствовать несколько, если на 1 нграму приходится слишком много сущностей в Викиданных. 
### 

In [11]:
from math import ceil
def parting(xs, parts): # функция делит список на указанное число частей (если у нас слишком много Q на нграмму)
    part_len = ceil(len(xs)/parts)
    return [xs[part_len*k:part_len*(k+1)] for k in range(parts)]

In [12]:
# разделим номера Q на части для тех нграм, которым соответствует слишком много сущностей (чтобы запрос что-то выдавал)
# попробуем построить запросы без ограничений
# для этого преобразуем значения словаря в формат для SPARQL

for k,v in unigrams_with_Q.items():
    v = ["(wd:{})".format(Q) for Q in list(v)]
    
    if len(v) <= 10:
        unigrams_with_Q[k] = [' '.join(v)]
    else:
        parts = int(str(len(v))[0]) + 1
        unigrams_with_Q[k] = [' '.join(i) for i in parting(v, parts)]

In [13]:
unigrams_with_Q["родник"] # такими порциями мы будем строить SPARQL-запросы для униграмы "родник" - 7 запросов

['(wd:Q15268365) (wd:Q27582223) (wd:Q28667634) (wd:Q4395919) (wd:Q18799045) (wd:Q1040258) (wd:Q4395930) (wd:Q31466546) (wd:Q19686707) (wd:Q19216505)',
 '(wd:Q15254709) (wd:Q22344134) (wd:Q56311162) (wd:Q4395926) (wd:Q23958762) (wd:Q13206716) (wd:Q1063652) (wd:Q13640673) (wd:Q47045425) (wd:Q19216503)',
 '(wd:Q20674388) (wd:Q31649769) (wd:Q29627438) (wd:Q21200255) (wd:Q19923038) (wd:Q19805535) (wd:Q37958593) (wd:Q25609143) (wd:Q124714) (wd:Q18799041)',
 '(wd:Q18799046) (wd:Q19216508) (wd:Q3939909) (wd:Q4395915) (wd:Q25606887) (wd:Q15268481) (wd:Q1250518) (wd:Q15255015) (wd:Q4395922) (wd:Q19805537)',
 '(wd:Q16692299) (wd:Q20144521) (wd:Q1054158) (wd:Q56307313) (wd:Q18799042) (wd:Q5767074) (wd:Q4395910) (wd:Q19691034) (wd:Q21343024) (wd:Q37958590)',
 '(wd:Q4395916) (wd:Q31694933) (wd:Q31661955) (wd:Q4395911) (wd:Q4395913) (wd:Q22344136) (wd:Q4395918) (wd:Q4395923) (wd:Q1509176) (wd:Q4395921)',
 '(wd:Q193318) (wd:Q4395920) (wd:Q13643657) (wd:Q21631844) (wd:Q1064948)']

### Напишем функции запросов

In [14]:
#http://tinyurl.com/y9m738lr  - простые триплеты с датой
def getSimpleTriplets(unigram, q_numbers):
    
    url = "https://query.wikidata.org/sparql"
    
    query = """SELECT ?item ?itemLabel ?predicate ?propertyLabel ?date
    WHERE
    {
      VALUES (?item) {(...)}
  
      ?item ?predicate ?date. 
      FILTER(DATATYPE(?date) = xsd:dateTime).
      MINUS { ?item schema:dateModified ?date. }
      MINUS { ?item wikibase:timestamp ?date. }
  
      BIND( IRI(REPLACE( STR(?predicate),"prop/direct/","entity/" )) AS ?property). 
      SERVICE wikibase:label { bd:serviceParam wikibase:language "ru, en". }
    }
    ORDER BY ?item ?date
    """
        
    query = query.replace('(...)', q_numbers)
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()

    dates = []
    
    if data != []:
        for item in data['results']['bindings']:
            dates.append(OrderedDict({
                'Unigram' : unigram,
                '#Q':  item['item']['value'].split("/")[-1],
                'Subject': item['itemLabel']['value'],
                '#P': item['predicate']['value'].split("/")[-1],
                'Predicate': item['propertyLabel']['value'],
                'Object': '',
                'Organization': '',
                'Just Date': str(item['date']['value'])[:-10],
                'Time Point' : '',
                'Start Time' : '',
                'End Time' : ''})) 
            
    if dates != []:
        df = pd.DataFrame(dates)
        return df



#http://tinyurl.com/yboysd2z  - start time-end time
def getStartEnd(unigram, q_numbers):
    
    url = "https://query.wikidata.org/sparql"
    
    query = """SELECT ?item ?itemLabel ?ps ?propertyLabel ?objectLabel ?organizationLabel ?starttime ?endtime
    WHERE {
      VALUES (?item) {(...)}
  
      ?item ?predicate ?statement .  
      ?statement ?ps ?object. 
      OPTIONAL {?statement pq:P642 ?organization}.
      ?statement pq:P580 ?starttime.
      OPTIONAL {?statement pq:P582 ?endtime}. 
  
      # ps:P1429 - ? - питомец
      FILTER(?ps NOT IN ( ps:P1448, ps:P395, ps:P190, ps:P1082, 
                          ps:P155, ps:P26, ps:P102, ps:P551, 
                          ps:P2131, ps:P2573, ps:P1081, ps:P2250,
                          ps:P4841, ps:P1279, ps:P172, ps:P5167, 
                          ps:P4010, ps:P2299, ps:P5982, ps:P2132, 
                          ps:P2134, ps:P2219, ps:P1198, ps:P3000,
                          ps:P41, ps:P2397, ps:P2002, ps:P27,
                          ps:P1037, ps:P108, ps:P1038, ps:P18,
                          ps:P185, ps:P451, ps:P1174, ps:P1128, 
                          ps:P2046, ps:P1114, ps:P1539, ps:P1540, 
                          ps:P3087, ps:P2403, ps:P1831, ps:P2325, 
                          ps:P195, ps:P1538, ps:P305, ps:P3872, 
                          ps:P185, ps:P1098, ps:P2139, ps:P1128, 
                          ps:P421
                          ) )
      ?property wikibase:statementProperty ?ps .
  
      SERVICE wikibase:label { bd:serviceParam wikibase:language "ru, en" }
    }
    ORDER BY ?itemLabel ?propertyLabel
    """
    
    query = query.replace('(...)', q_numbers)
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    
    dates = []
    
    if data != []:
        for item in data['results']['bindings']:
            dates.append(OrderedDict({
                'Unigram' : unigram,
                '#Q':  item['item']['value'].split("/")[-1],
                'Subject': item['itemLabel']['value'],
                '#P': item['ps']['value'].split("/")[-1],
                'Predicate': item['propertyLabel']['value'],
                'Object': item['objectLabel']['value'],
                'Organization': item['organizationLabel']['value'] if 'organizationLabel' in item else '',
                'Just Date': '',
                'Time Point' : '',
                'Start Time' : str(item['starttime']['value'])[:-10],
                'End Time' : str(item['endtime']['value'])[:-10] if 'endtime' in item else ''
            })) 
            
    if dates != []:
        df = pd.DataFrame(dates)
        return df


#http://tinyurl.com/y947qncg  -　point in time
def getTimePoint(unigram, q_numbers):
    
    url = "https://query.wikidata.org/sparql"
    
    query = """SELECT ?item ?itemLabel ?property ?propertyLabel ?objectLabel ?organizationLabel ?point_in_time
    WHERE {
      VALUES (?item) {(...)}
  
      ?item ?predicate ?statement .  
      ?statement ?ps ?object . 
      OPTIONAL {?statement pq:P642 ?organization}.
      ?statement pq:P585 ?point_in_time.
  
      FILTER(?ps NOT IN ( ps:P1448, ps:P395, ps:P190, ps:P1082, 
                          ps:P155, ps:P26, ps:P102, ps:P551, 
                          ps:P2131, ps:P2573, ps:P1081, ps:P2250,
                          ps:P4841, ps:P1279, ps:P172, ps:P5167, 
                          ps:P4010, ps:P2299, ps:P5982, ps:P2132, 
                          ps:P2134, ps:P2219, ps:P1198, ps:P3000,
                          ps:P41, ps:P2397, ps:P2002, ps:P27,
                          ps:P1037, ps:P108, ps:P1038, ps:P18,
                          ps:P185, ps:P451, ps:P1174, ps:P1128, 
                          ps:P2046, ps:P1114, ps:P1539, ps:P1540, 
                          ps:P3087, ps:P2403, ps:P1831, ps:P2325, 
                          ps:P195, ps:P1538, ps:P305, ps:P3872, 
                          ps:P185, ps:P1098, ps:P2139, ps:P1128, 
                          ps:P421
                          ) )
  
      ?property wikibase:statementProperty ?ps .
  
      SERVICE wikibase:label { bd:serviceParam wikibase:language "ru,en" }
    }

    ORDER BY ?itemLabel ?propertyLabel

    """
    
    query = query.replace('(...)',q_numbers)
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    
    dates = []
    
    if data != []:
        for item in data['results']['bindings']:
            dates.append(OrderedDict({
                'Unigram' : unigram,
                '#Q':  item['item']['value'].split("/")[-1],
                'Subject': item['itemLabel']['value'],
                '#P': item['property']['value'].split("/")[-1],
                'Predicate': item['propertyLabel']['value'],
                'Object': item['objectLabel']['value'],
                'Organization': item['organizationLabel']['value'] if 'organizationLabel' in item else '',
                'Just Date': '',
                'Time Point' : str(item['point_in_time']['value'])[:-10],
                'Start Time' : '',
                'End Time' : ''
            })) 
            
    if dates != []:
        df = pd.DataFrame(dates)
        return df

In [15]:
results = pd.DataFrame(columns = ["Unigram", "#Q", "Subject", "#P", "Predicate", "Object", "Organization", 
                                  "Just Date","Time Point", "Start Time", "End Time"])
results.head()

,Unigram,#Q,Subject,#P,Predicate,Object,Organization,Just Date,Time Point,Start Time,End Time


In [16]:
for k, v in unigrams_with_Q.items():
    for qs in v:
        results = pd.concat([results, getSimpleTriplets(k, qs)])
        results = pd.concat([results, getStartEnd(k, qs)])
        results = pd.concat([results, getTimePoint(k, qs)])

KeyboardInterrupt: 

### Вот так будет выглядеть итоговый датафрейм, который я позднее сохраню в csv для каждой буквы без индексов. 

In [17]:
results

,Unigram,#Q,Subject,#P,Predicate,Object,Organization,Just Date,Time Point,Start Time,End Time
0,рааг,Q2639948,"Рааг, Ильмар",P569,дата рождения,,,1968-05-21,,,
0,раазик,Q1432542,Раазику,P580,дата начала,,,1992-03-12,,,
1,раазик,Q2097814,Раазику,P1249,первое письменное упоминание,,,1393-01-01,,,
0,раамат,Q3742252,"Раамат, Рейн Аугустович",P569,дата рождения,,,1931-03-20,,,
0,раанан,Q309164,Раанана,P571,дата основания/создания/возникновения,,,1921-01-01,,,
1,раанан,Q309164,Раанана,P571,дата основания/создания/возникновения,,,1922-04-02,,,
0,раасна,Q4397939,Российская академия архитектуры и строительных...,P571,дата основания/создания/возникновения,,,1992-01-01,,,
0,рааха,Q622592,Раахе,P571,дата основания/создания/возникновения,,,1649-01-01,,,
0,раб,Q695414,Раба,P131,административно-территориальная единица,Raaba-Grambach,,,,2015-01-01,
0,раба,Q141607,(1624) Рабе,P575,дата открытия,,,1931-10-09,,,


In [64]:
k = ['P185', 'P451', 'P1174', 'P1128', 'P2046', 'P1114', 'P1539', 'P1540', 'P3087', 'P2403', 'P1831', 'P2325', 'P195',
     'P1538', 'P305', 'P3872', 'P185', 'P1098', 'P2139', 'P1128', 'P421']

k = ', '.join(["ps:" + i for i in k])
     
# совокупные активы - пока удалено
# число сотрудников - пока удалено